In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [2]:
import pytesseract
from PIL import Image

In [4]:
from google.colab import files
uploaded = files.upload()

Saving Screenshot (214).png to Screenshot (214).png


In [3]:
image_path_in_colab='Screenshot (214).png'
article_text = pytesseract.image_to_string(Image.open(image_path_in_colab))
print(article_text)

It was a hot summer day. A lion and a boar reach a
small water body for a drink. They begin arguing
and fighting about who should drink first. After a
while, they are tired and stop for breath, when they
notice vultures above. Soon they realize that the
vultures are waiting for one or both of them to fall,
to feast on them. The lion and the boar then decide
that it was best to make up and be friends than
fight and become food for vultures. They drink the
water together and go their ways after.



## **Extractive Summarizarion using TextRank algorithm**

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

import pandas as pd
import unicodedata
import numpy as np
import string

from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
def read_article(artcle_text):
    article = article_text.split(". ")
    sentences = []

    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

In [ ]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [ ]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

In [ ]:
def generate_summary(article_text, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(article_text)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    #print(". ".join(summarize_text))
    summary = ". ".join(summarize_text)
    return summary

In [ ]:
summary = generate_summary(article_text, 1)
print(summary)

Wisconsin broke records this week for new coronavirus cases, deaths and
hospitalizations, leading to the opening of a field hospital to handle COVID-19
patients
Gov
Tony Evers said he plans to activate the Wisconsin National
Guard to fill any staffing shortages at election sites.

While holding a competitive presidential election during a pandemic is “tricky
business,” the governor said, “People are ready to have this election over, and I
think it will be a successful election with very few hiccups."

In Iowa, Scott County Auditor Roxanna Moritz opened additional early voting
sites in and around Davenport, the state's third-largest city, to try to reduce
the number people casting ballots on Election Day and to keep the virus from
spreading in large precincts.

Indexes of top ranked_sentence order are  [(0.5, ['Wisconsin', 'broke', 'records', 'this', 'week', 'for', 'new', 'coronavirus', 'cases,', 'deaths', 'and\nhospitalizations,', 'leading', 'to', 'the', 'opening', 'of', 'a', 'field',

# **Abstractive Summarization**

In [5]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from collections import Counter
import os

import Summarizer
import summarizer_data_utils
import summarizer_model_utils

In [7]:
print(tf.__version__)

1.15.2


In [8]:
data = pd.read_csv('./drive/My Drive/text_summary_data/articles1.csv',
                   encoding='utf-8')
data1 = pd.read_csv('./drive/My Drive/text_summary_data/articles2.csv',
                    encoding='utf-8')
data2 = pd.read_csv('./drive/My Drive/text_summary_data/articles3.csv',
                    encoding='utf-8')

In [9]:
data = pd.concat([data, data1, data2])
data.shape

(142570, 10)

In [11]:
data.head(10)

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."
5,5,17288,"Sick With a Cold, Queen Elizabeth Misses New Y...",New York Times,Sewell Chan,2017-01-02,2017.0,1.0,NaN,"LONDON — Queen Elizabeth II, who has been b..."
6,6,17289,Taiwan’s President Accuses China of Renewed In...,New York Times,Javier C. Hernández,2017-01-02,2017.0,1.0,NaN,BEIJING — President Tsai of Taiwan sharpl...
7,7,17290,"After ‘The Biggest Loser,’ Their Bodies Fought...",New York Times,Gina Kolata,2017-02-08,2017.0,2.0,NaN,"Danny Cahill stood, slightly dazed, in a blizz..."
8,8,17291,"First, a Mixtape. Then a Romance. - The New Yo...",New York Times,Katherine Rosman,2016-12-31,2016.0,12.0,NaN,"Just how is Hillary Kerr, the founder of ..."
9,9,17292,Calling on Angels While Enduring the Trials of...,New York Times,Andy Newman,2016-12-31,2016.0,12.0,NaN,Angels are everywhere in the Muñiz family’s ap...


In [10]:
Counter(data.publication)

Counter({'Atlantic': 7179,
         'Breitbart': 23781,
         'Business Insider': 6757,
         'Buzzfeed News': 4854,
         'CNN': 11488,
         'Fox News': 4354,
         'Guardian': 8681,
         'NPR': 11992,
         'National Review': 6203,
         'New York Post': 17493,
         'New York Times': 7803,
         'Reuters': 10710,
         'Talking Points Memo': 5214,
         'Vox': 4947,
         'Washington Post': 11114})

In [11]:
# Two titles are missing. 
data.isnull().sum()

Unnamed: 0         0
id                 0
title              2
publication        0
author         15876
date            2641
year            2641
month           2641
url            57011
content            0
dtype: int64

In [12]:
# drop those. 
data.dropna(subset=['title'], inplace = True)

In [13]:
# to make the transition from the amazon review example to this one as comfortable as possbile we just rename 
# the columns. 
data.rename(index = str, columns = {'title':'Summary', 'content':'Text'}, inplace = True)
data = data[['Summary', 'Text']]

In [14]:
data = data[['Summary', 'Text']]
data.head()

,Summary,Text
0,House Republicans Fret About Winning Their Hea...,WASHINGTON — Congressional Republicans have...
1,Rift Between Officers and Residents as Killing...,"After the bullet shells get counted, the blood..."
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...","When Walt Disney’s “Bambi” opened in 1942, cri..."
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...","Death may be the great equalizer, but it isn’t..."
4,Kim Jong-un Says North Korea Is Preparing to T...,"SEOUL, South Korea — North Korea’s leader, ..."


In [15]:
data.shape

(142568, 2)

In [16]:
len_summaries = [len(summary) for i, summary in enumerate(data.Summary)]
len_texts = [len(text) for text in data.Text]

In [17]:
len_summaries_counted = Counter(len_summaries).most_common()
len_texts_counted = Counter(len_texts).most_common()
len_summaries_counted[:10], len_texts_counted[:10]

([(63, 3512),
  (64, 3451),
  (65, 3364),
  (62, 3362),
  (60, 3344),
  (59, 3310),
  (61, 3284),
  (58, 3280),
  (67, 3016),
  (66, 3015)],
 [(2878, 44),
  (13, 42),
  (1742, 41),
  (2457, 41),
  (1738, 41),
  (2199, 40),
  (2673, 40),
  (1548, 40),
  (2617, 40),
  (2913, 39)])

In [18]:
indices = [ind for ind, text in enumerate(data.Text) if 50 < len(text) < 700]

In [19]:
len(indices)

5037

In [20]:
texts_unprocessed = []
summaries_unprocessed = []

for text, summary in zip(data.Text, data.Summary):
    if 50< len(text) < 700:
        texts_unprocessed.append(text)
        summaries_unprocessed.append(summary)

In [21]:
len(indices), len(texts_unprocessed), len(summaries_unprocessed)

(5037, 5037, 5037)

In [22]:
to_remove = ['- The New York Times', '- Breitbart']

summaries_unprocessed_clean = []
texts_unprocessed_clean = []

removed = 0
append = True
for sentence in summaries_unprocessed:
    append = True
    for r in to_remove:
        if sentence.endswith(r):
            sentence = sentence.replace(r, '.')
            summaries_unprocessed_clean.append(sentence.replace(r, '.'))
            removed+=1
            append = False
            break
            
    if append:
        summaries_unprocessed_clean.append(sentence)

In [23]:
len(summaries_unprocessed_clean)

5037

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
processed_texts, processed_summaries, words_counted = summarizer_data_utils.preprocess_texts_and_summaries(
    texts_unprocessed,
    summaries_unprocessed_clean,
    keep_most=False)

Processing Time:  4.182254076004028


In [25]:
processed_texts_clean = []
processed_summaries_clean = []

for t, s in zip(processed_texts, processed_summaries):
    if t != [] and s != []:
        processed_texts_clean.append(t)
        processed_summaries_clean.append(s)

In [26]:
specials = ["<EOS>", "<SOS>","<PAD>","<UNK>"]
word2ind, ind2word,  missing_words = summarizer_data_utils.create_word_inds_dicts(words_counted,
                                                                                  specials = specials,
                                                                                  min_occurences = 2)
print(len(word2ind), len(ind2word), len(missing_words))

14299 14299 9809


In [29]:
print(tf.__version__)

1.15.2


In [27]:
# embed = hub.Module("https://tfhub.dev/google/nnlm-en-dim128/1")
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")
emb = embed([key for key in word2ind.keys()])

with tf.compat.v1.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    embedding = sess.run(emb)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [28]:
embedding.shape

(14299, 250)

In [29]:
np.save('./tf_hub_embedding_headlines.npy', embedding)

In [30]:
# converts words in texts and summaries to indices
converted_texts, unknown_words_in_texts = summarizer_data_utils.convert_to_inds(processed_texts_clean,
                                                                                word2ind,
                                                                                eos = False)

In [31]:
converted_summaries, unknown_words_in_summaries = summarizer_data_utils.convert_to_inds(processed_summaries_clean,
                                                                                        word2ind,
                                                                                        eos = True,
                                                                                        sos = True)

In [32]:
len(converted_texts), len(converted_summaries)

(5034, 5034)

In [33]:
print(summarizer_data_utils.convert_inds_to_text(converted_texts[0], ind2word))
print(summarizer_data_utils.convert_inds_to_text(converted_summaries[0], ind2word))

['the', 'reaction', 'on', 'social', 'media', 'to', 'the', 'news', 'that', 'beyonc', 'is', 'pregnant', 'with', 'twins', 'was', 'by', 'turns', 'swift', 'sweet', 'and', 'strange', 'while', 'beyonce', 's', 'super', 'bowl', 'performance', 'of', 'the', 'hit', 'song', 'formation', 'was', 'seen', 'as', 'a', 'pointed', 'political', 'statement', 'addressing', 'issues', 'like', 'black', 'pride', 'and', 'police', 'brutality', 'it', 'and', 'the', 'video', 'drew', 'anger', 'from', 'some', 'in', 'law', 'enforcement', 'who', 'said', 'they', 'were', 'an', 'attack', 'on', 'their', 'profession', 'then', 'there', 'were', 'others', 'like', 'the', 'police', 'officer', 'in', 'virginia', 'who', '<UNK>', 'her', '<UNK>', '<UNK>', 'to', 'inspire', 'high', 'school', 'students', 'a', 'phone', 'call', 'to', 'the', 'atlanta', 'department', 'seeking', 'comment', 'was', 'not', 'immediately', 'returned', 'on', 'wednesday']
['<SOS>', 'beyonc', 'is', 'pregnant', 'and', 'twitter', 'loses', 'it', '<EOS>']


In [34]:
# model hyperparameters
num_layers_encoder = 4
num_layers_decoder = 4
rnn_size_encoder = 300
rnn_size_decoder = 300

batch_size = 32
epochs = 100
clip = 5
keep_probability = 0.8
learning_rate = 0.0005
max_lr=0.005
learning_rate_decay_steps = 100
learning_rate_decay = 0.90


pretrained_embeddings_path = './tf_hub_embedding_headlines.npy'
summary_dir = os.path.join('./tensorboard/headlines')

use_cyclic_lr = True
inference_targets=True

In [ ]:
# build graph and train the model 
summarizer_model_utils.reset_graph()
summarizer = Summarizer.Summarizer(word2ind,
                                   ind2word,
                                   save_path='./drive/My Drive/models/headlines/larger_model',
                                   mode='TRAIN',
                                   num_layers_encoder = num_layers_encoder,
                                   num_layers_decoder = num_layers_decoder,
                                   rnn_size_encoder = rnn_size_encoder,
                                   rnn_size_decoder = rnn_size_decoder,
                                   batch_size = batch_size,
                                   clip = clip,
                                   keep_probability = keep_probability,
                                   learning_rate = learning_rate,
                                   max_lr=max_lr,
                                   learning_rate_decay_steps = learning_rate_decay_steps,
                                   learning_rate_decay = learning_rate_decay,
                                   epochs = epochs,
                                   pretrained_embeddings_path = pretrained_embeddings_path,
                                   use_cyclic_lr = use_cyclic_lr,)
#                                    summary_dir = summary_dir)           

summarizer.build_graph()
summarizer.train(converted_texts, 
                 converted_summaries)

In [35]:
# converts words in texts and summaries to indices
tokenized_text = summarizer_data_utils.preprocess(
    article_text,
    keep_most=False)
tokenized_text = [tokenized_text]
converted_texts, unknown_words_in_texts = summarizer_data_utils.convert_to_inds(tokenized_text,
                                                                                word2ind,
                                                                                eos = False)

In [36]:
summarizer_model_utils.reset_graph()
summarizer = Summarizer.Summarizer(word2ind,
                                   ind2word,
                                   './drive/My Drive/models/headlines/larger_model',
                                   'INFER',
                                   num_layers_encoder = num_layers_encoder,
                                   num_layers_decoder = num_layers_decoder,
                                   batch_size = len(converted_texts[:1]),
                                   clip = clip,
                                   keep_probability = 1.0,
                                   learning_rate = 0.0,
                                   beam_width = 5,
                                   rnn_size_encoder = rnn_size_encoder,
                                   rnn_size_decoder = rnn_size_decoder,
                                   inference_targets = False,
                                   pretrained_embeddings_path = pretrained_embeddings_path)
summarizer.build_graph()
preds = summarizer.infer(converted_texts[:1],
                         restore_path =  './drive/My Drive/models/headlines/larger_model')

Loaded pretrained embeddings.



Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API


...

Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


Graph built.
INFO:tensorflow:Restoring parameters from ./drive/My Drive/models/headlines/larger_model


INFO:tensorflow:Restoring parameters from ./drive/My Drive/models/headlines/larger_model


Done.


In [37]:
summarizer_model_utils.sample_results(preds,
                                      ind2word,
                                      word2ind,
                                      converted_texts[:1])




 ----------------------------------------------------------------------------------------------------
Actual Text:
it was a hot summer day a <UNK> and a <UNK> reach a small water body for a drink they begin arguing and fighting about who should drink first after a while they are tired and stop for breath when they notice <UNK> above soon they realize that the <UNK> are waiting for one or both of them to fall to feast on them the <UNK> and the <UNK> then decide that it was best to make up and be friends than fight and become food for <UNK> they drink the water together and go their ways after

Created Summary:
how actually day us at the looked the awesome




## **Text-to-Speech**

In [38]:
!pip3 install gTTS pyttsx3 playsound

  Created wheel for gtts-token: filename=gTTS_token-1.1.3-cp36-none-any.whl size=4096 sha256=7264c851da65c232439871160415cada5be706fca55c6d632aaf985c8a7a42c1
  Stored in directory: /root/.cache/pip/wheels/dd/11/61/33f7e51bf545e910552b2255eead2a7cd8ef54064b46dceb34
Successfully built gtts-token


In [39]:
import gtts
from playsound import playsound

In [40]:
tts = gtts.gTTS(summary)
tts.save("summary.mp3")